In [1]:
import time
import random
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

# Load data

In [2]:
df = pd.read_csv("passport_usps.csv")
df.head()

,Name,Type,Address,Phone,PhotoOnSite,Comments,Distance,url,domain,phone,NeedsAppointment,zip
0,USPS JOSE MARTI STATION,Postal,"425 NW 27TH AVE MIAMI, FL 33125",3055412782,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,3,https://usps.com/scheduler,usps,NaN,True,33125
1,USPS MIAMI BEACH MAIN POST OFFICE,Postal,"1300 WASHINGTON AVENUE MIAMI BEACH, FL 33139",3055385187,Yes,***BY APPOINTMENT ONLY*** 305 672-2447 To sche...,3,https://usps.com/scheduler,usps,305-672-2447,True,33139
2,USPS LITTLE RIVER BRANCH,Postal,"140 NE 84TH ST MIAMI, FL 33138",3057541906,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,4,https://usps.com/scheduler,usps,NaN,True,33138
3,USPS OCEAN VIEW POST OFFICE,Postal,"445 W 40TH ST MIAMI BEACH, FL 33140",3055386536,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,4,https://usps.com/scheduler,usps,NaN,True,33140
4,USPS HIALEAH MAIN POST OFFICE,Postal,"325 E 1ST AVE HIALEAH, FL 33010",3058895421,Yes,***By Appointments** Visit Website: https://ww...,6,https://usps.com/scheduler,usps,NaN,Unknown,33010


# Scrape for available facilities on June 18

In [3]:
link = 'https://tools.usps.com/rcas.htm'

def get_availability(current_zip):
    driver = webdriver.Chrome(executable_path='/Users/vrai/Downloads/chromedriver')
    driver.get(link)
    driver.implicitly_wait(30)
    
    # Service type
    Select(driver.find_element_by_id("passportappointmentType")).select_by_value('newpassport')
    driver.implicitly_wait(10)
    
    # How many adults
    Select(driver.find_element_by_id("adultCount")).select_by_value('1')
    driver.implicitly_wait(10)
    
    # Search by date
    search_by = driver.find_element_b# Load datay_id("cb-date")
    search_by.click()
    driver.implicitly_wait(10)
    
    # Click on calendar
    calendar = driver.find_element_by_id("calendarpop")
    calendar.click()
    driver.implicitly_wait(10)
    
    try:
        # Select June 18
        my_date = driver.find_element(By.ID, 'scheduleDatePicker').find_elements(By.TAG_NAME, 'tr')[3].find_elements(By.TAG_NAME, 'td')[5]
        my_date.click()
        driver.implicitly_wait(10)

        # Select date
        select_date = driver.find_element_by_id("selectDateFromModal")
        select_date.click()
        driver.implicitly_wait(10)

        # Write zip code
        zip_code = driver.find_element_by_id("SEARCH_ZIP5")
        zip_code.send_keys(current_zip)
        driver.implicitly_wait(10)

        # Search within one mile
        Select(driver.find_element_by_id("radius")).select_by_value('1')
        driver.implicitly_wait(10)

        # Search facility
        facility = driver.find_element_by_id("searchFacilitiesButton")
        facility.click()
        driver.implicitly_wait(10)

        # Obtain availability
        result = " ".join([e.text for e in driver.find_element_by_id("searchResultList").find_elements(By.TAG_NAME, 'div')[-2].find_elements(By.TAG_NAME, 'span')])

    except:
        result = None
        
    pause = random.randrange(100) / 100.0
    time.sleep(pause)
    
    driver.close()
    
    return result

In [4]:
df['Jun18'] = df.zip.apply(lambda current_zip: get_availability(current_zip))

### Inner loop

In [17]:
df['Verificar'] = df['Jun18'].apply(lambda val: True if "JUN 18" in str(val) else False)

In [19]:
df_done = df[df.Verificar]
df_todo = df[~df.Verificar]

In [39]:
df_todo['Jun18'] = df_todo.zip.apply(lambda current_zip: get_availability(current_zip))

<ipython-input-39-fa7724593cbe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_todo['Jun18'] = df_todo.zip.apply(lambda current_zip: get_availability(current_zip))


In [40]:
df_todo['Verificar'] = df_todo['Jun18'].apply(lambda val: True if "JUN 18" in str(val) else False)
df_todo['Verificar'].value_counts()

<ipython-input-40-767213eb6feb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_todo['Verificar'] = df_todo['Jun18'].apply(lambda val: True if "JUN 18" in str(val) else False)


True    1
Name: Verificar, dtype: int64

In [41]:
df_done = pd.concat([df_done,df_todo[df_todo['Verificar']]])
df_todo = df_todo[~df_todo.Verificar]

In [48]:
assert df_done.shape[0] == df.shape[0]
assert df_todo.shape[0] == 0

### Explore results

In [58]:
df_final = df_done.copy()
df_final = df_final.sort_index()
df_final.drop(['Verificar', 'zip', 'url', 'domain', 'NeedsAppointment'], axis=1, inplace=True)

In [60]:
df_final['OK'] = df_final.Jun18.apply(lambda val: True if val=="JUN 18" else False)

In [62]:
df_final['OK'].value_counts()

False    77
True     44
Name: OK, dtype: int64

In [63]:
df_final[df_final['OK']]

,Name,Type,Address,Phone,PhotoOnSite,Comments,Distance,phone,Jun18,OK
28,USPS TAVERNIER POST OFFICE,Postal,"91220 OVERSEAS HWY TAVERNIER, FL 33070",3058531230,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,57,NaN,JUN 18,True
34,USPS ISLAMORADA POST OFFICE,Postal,"82801 OVERSEAS HWY ISLAMORADA, FL 33036",3056644738,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,71,NaN,JUN 18,True
36,USPS INDIANTOWN POST OFFICE,Postal,"15300 SW ADAMS AVE INDIANTOWN, FL 34956",7725972406,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,93,NaN,JUN 18,True
37,USPS JENSEN BEACH POST OFFICE,Postal,"2301 NE SAVANNAH RD JENSEN BEACH, FL 34957",7722320869,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,100,NaN,JUN 18,True
38,USPS BONITA SPRINGS POST OFFICE,Postal,"26150 OLD 41ST ROAD BONITA SPRINGS, FL 34135",2399929742,Yes,"***BY APPOINTMENT ONLY*** For appointments, vi...",104,NaN,JUN 18,True
39,USPS BIG PINE KEY POST OFFICE,Postal,"29959 OVERSEAS HWY BIG PINE KEY, FL 33043",3058722531,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,105,NaN,JUN 18,True
40,USPS LEHIGH ACRES POST OFFICE,Postal,"1201 BUSINESS WAY LEHIGH ACRES, FL 33936",2393698911,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,105,NaN,JUN 18,True
41,USPS ALVA POST OFFICE,Postal,"17301 BROADWAY ST ALVA, FL 33920",2397283924,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,109,NaN,JUN 18,True
42,USPS FORT PIERCE MAIN POST OFFICE,Postal,"5000 W MIDWAY RD FORT PIERCE, FL 34981",7724600841,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,111,855-864-3379,JUN 18,True
43,USPS SUMMERLAND KEY POST OFFICE,Postal,"24700 OVERSEAS HWY SUMMERLAND KEY, FL 33042",3057453391,Yes,***BY APPOINTMENT ONLY*** To schedule an appoi...,115,NaN,JUN 18,True


In [64]:
df_final.to_csv("passport_USPS_final.csv", index=False)